## Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import seaborn as sns
import yaml

from apex_ocr.config import DATABASE_YML_FILE
from apex_ocr.database.api import ApexDatabaseApi
from apex_ocr.database.models import MatchResult, Player, PlayerMatchResult

In [ ]:
pd.options.display.float_format = '{:,.3f}'.format
%matplotlib ipympl


## Database Connection

In [ ]:
with open(DATABASE_YML_FILE) as db_file:
    db_config = yaml.load(db_file, Loader=yaml.FullLoader)

dialect = db_config["dialect"]
username = db_config["username"]
password = db_config["password"]
hostname = db_config["hostname"]
port = db_config["port"]
database_name = db_config["database_name"]

db_conn_str = f"{dialect}://{username}:{password}@{hostname}:{port}/{database_name}"
db_conn = ApexDatabaseApi(db_conn_str)

## Group vs Solo Result Comparison

In [ ]:
ccc_match_results = []
red_ninja_match_results = []
redlezar_solo_results = []
prodigal_solo_results = []
fourransom_solo_results = []

ignore_cols = [
    "id",
    "match_id",
    "datetime",
    "match_type",
    "result",
    "hash",
    "player_id",
    "clan_id",
    "player_name",
    "legend",
]

match_results = db_conn.session.query(MatchResult).all()

for match_result in match_results:
    player_names = set([pmr.player.name for pmr in match_result.player_match_results])

    if set(["redlezar", "4ransom", "prodigalninja"]) == player_names:
        ccc_match_results.append(match_result)
    elif set(["redlezar", "prodigalninja"]).issubset(player_names):
        red_ninja_match_results.append(match_result)
    elif "redlezar" in player_names:
        redlezar_solo_results.append(match_result)
    elif "prodigalninja" in player_names:
        prodigal_solo_results.append(match_result)
    elif "4ransom" in player_names:
        fourransom_solo_results.append(match_result)

In [ ]:
full_match_results = pd.DataFrame()

for match_result in ccc_match_results:
    for pmr in match_result.player_match_results:
        temp_df = pd.DataFrame(pmr.to_dict(), index=[0])
        temp_df["group"] = "ccc"
        full_match_results = pd.concat([full_match_results, temp_df], ignore_index=True)

for match_result in red_ninja_match_results:
    for pmr in match_result.player_match_results:
        temp_df = pd.DataFrame(pmr.to_dict(), index=[0])
        temp_df["group"] = "redninja"
        full_match_results = pd.concat([full_match_results, temp_df], ignore_index=True)

for match_result in redlezar_solo_results:
    for pmr in match_result.player_match_results:
        temp_df = pd.DataFrame(pmr.to_dict(), index=[0])
        temp_df["group"] = "redsolo"
        full_match_results = pd.concat([full_match_results, temp_df], ignore_index=True)

for match_result in prodigal_solo_results:
    for pmr in match_result.player_match_results:
        temp_df = pd.DataFrame(pmr.to_dict(), index=[0])
        temp_df["group"] = "ninjasolo"
        full_match_results = pd.concat([full_match_results, temp_df], ignore_index=True)

full_match_results.drop(columns=ignore_cols).groupby(["group"]).mean()

## All Player Match Results Statistics

In [ ]:
player_match_results = pd.DataFrame()
for pmr in db_conn.session.query(PlayerMatchResult).all():
    player_match_results = pd.concat(
        [player_match_results, pd.DataFrame(pmr.to_dict(), index=[0])]
    )

pmr_counts = player_match_results.player_name.value_counts()
pmr_counts


## Compare Individual Players

In [ ]:
prodigalninja = db_conn.session.query(Player).filter_by(name="prodigalninja").first()
redlezar = db_conn.session.query(Player).filter_by(name="redlezar").first()
fransom = db_conn.session.query(Player).filter_by(name="4ransom").first()
lightningquick3 = db_conn.session.query(Player).filter_by(name="lightningquick3").first()
laxman117 = db_conn.session.query(Player).filter_by(name="laxman117").first()


In [ ]:
results_df = pd.DataFrame()

for match_result in prodigalninja.match_results:
    results_df = pd.concat(
        [results_df, pd.DataFrame(match_result.to_dict(), index=[0])]
    )

for match_result in redlezar.match_results:
    results_df = pd.concat(
        [results_df, pd.DataFrame(match_result.to_dict(), index=[0])]
    )

for match_result in fransom.match_results:
    results_df = pd.concat(
        [results_df, pd.DataFrame(match_result.to_dict(), index=[0])]
    )

for match_result in lightningquick3.match_results:
    results_df = pd.concat(
        [results_df, pd.DataFrame(match_result.to_dict(), index=[0])]
    )

for match_result in laxman117.match_results:
    results_df = pd.concat(
        [results_df, pd.DataFrame(match_result.to_dict(), index=[0])]
    )

# Filter out bad data
results_df = results_df[results_df["place"] <= 20]

results_df.head()

In [ ]:
pmr_mean = (
    results_df[
        [
            "place",
            "player_name",
            "kills",
            "assists",
            "knockdowns",
            "damage",
            "survival_time",
            "revives",
            "respawns",
        ]
    ]
    .groupby("player_name")
    .mean()
    .reset_index()
)
pmr_mean.insert(1, "count", pmr_mean["player_name"].apply(lambda v: pmr_counts[v]))
pmr_mean.insert(6, "efficiency", pmr_mean["kills"] / pmr_mean["knockdowns"])
pmr_mean

## Plots

In [ ]:
player_filter = ["4ransom", "redlezar", "prodigalninja"]


In [ ]:
plt.figure()
sns.histplot(
    data=results_df[results_df.player_name.isin(player_filter)],
    hue="player_name",
    x="place",
    discrete=True,
    element="step",
    common_norm=False,
    stat="density",
)

In [ ]:
plt.figure()
ax = sns.regplot(data=results_df, x="survival_time", y="place")
ax.set_title("Surival Time vs Place")
ax.set_xlabel("Surival Time (s)")
ax.set_ylabel("Place")
ax.set_ylim(20.5, 0)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%d"))
ax.xaxis.set_major_formatter(ticker.FormatStrFormatter("%2.2f"))